# Regresión lineal utilizando Tensorflow

## Predicción de precios de casas utilizando un modelo de regresión lineal simple


Un modelo de regresión lineal busca encontrar la ecuación lineal que minimice el error cuadrado medio.

La ecuación lineal:

$h(x) = wx + b$

donde: 

* $x$ : Variable independiente
* $w$ : Pendiente
* $b$ : Intercepto

Función de costo:

$C(w,b) = \frac{1}{2} \sum_{i=1}^{n} (y_i − h(x_i))^2$

donde:

* $y_i$ : Valor real de cada dato en el dataset
* $h(x_{i})$ : Valor predecido por el modelo

Para poder optimizar este modelo es necesario encontrar los valores de $w$ y $b$ que minimizan la función de costo.
Esto se realiza a través de un aprendizaje por medio de *gradient descent*.


In [10]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = np.load('proyecto_training_data.npy')


df = pd.DataFrame(data = df, columns = ["SalePrice", "OverallQual", "1stFlrSF", "TotRmsAbvGrd", "YearBuilt", "LotFrontage"])

#Información básica de dataframe
df.describe()

,SalePrice,OverallQual,1stFlrSF,TotRmsAbvGrd,YearBuilt,LotFrontage
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1201.000000
mean,180921.195890,6.099315,1162.626712,6.517808,1971.267808,70.049958
std,79442.502883,1.382997,386.587738,1.625393,30.202904,24.284752
min,34900.000000,1.000000,334.000000,2.000000,1872.000000,21.000000
25%,129975.000000,5.000000,882.000000,5.000000,1954.000000,59.000000
50%,163000.000000,6.000000,1087.000000,6.000000,1973.000000,69.000000
75%,214000.000000,7.000000,1391.250000,7.000000,2000.000000,80.000000
max,755000.000000,10.000000,4692.000000,14.000000,2010.000000,313.000000


In [12]:
# Split de datos de entrenamiento y test

#Random seed para reproducibilidad de los resultados

np.random.seed(42)

#Shuffle de datos
df = df.sample(frac = 1)

#Training and test split

train = int(0.8 * len(df))

dftrain = df[:train]
dftest = df[:-train]

print('Longitud de set de entrenamiento: ',len(dftrain), ', Longitud de set de prueba: ', len(dftest))


Longitud de set de entrenamiento:  1168 , Longitud de set de prueba:  292


1168

## Modelo de regresión lineal

### Variable a utilizar para modelo de regresión lineal

Según el análisis exploratorio de datos las variables que se utilizarán para crear los modelos de regresión lineal son:

* ```OverallQual``` con un $R = 0.79$

Se estará utilizando esta variable para la creación del modelo de regresión lineal.


### Entrenamiento de modelo de regresión lineal mediante ***gradient descent***

El modelo de regresión lineal se entrenará utilizando la siguiente función de costo. El algoritmo de gradient descent calcula la derivada parcial para cada uno de los parámetros de la función de costo. En cada iteración se realiza una modificación pequeña proporcional a esas derivadas parciales calculadas. A continuación se presenta este concepto en notación matemática.


Función de costo:

$C(w,b) = \frac{1}{2m} \sum_{i=1}^{m} (y_{i} − h(x_i))^2$

donde:

* $y_{i}$ : Valor real de cada dato en el dataset
* $wx_{i}+b$ : Valor predecido por el modelo

Derivadas parciales:

$\frac{\partial m }{\partial f} = \frac{1}{n} \sum_{i=1}^{n} (y_i − h(x_i))*m$

$\frac{\partial b }{\partial f} = \frac{1}{n} \sum_{i=1}^{n} (y_i − h(x_i))$

*Gradient descent*:

$w = w - \alpha \frac{1}{n} \sum_{i=1}^{n} (y_i - h(x_i))*m)$

$b = b - \alpha \frac{1}{n} \sum_{i=1}^{n} (y_i − h(x_i))$

$\alpha$ = tasa de aprendizaje

Los valores de $m$ y $b$ son actualizados iterativamente hasta minimizar el error, en otras palabras, hasta encontrar la línea de regresión que minimiza el error entre los datos predecidos y los datos reales.


### Implementación de modelo de regresión lineal con Tensorflow




In [203]:
class LinearModel:
    def __init__ (self):
        tf.reset_default_graph()
        self.w = tf.get_variable("weights", dtype = tf.float32, shape = [1,2], initializer = tf.zeros_initializer())
        
        
    def __call__(self, x):
        return tf.matmul(self.w,x)
    
    def update(self, x, y, learningrate):
        prediction = self(x)
        error = 1/2 * tf.reduce_mean(tf.math.square(y - prediction))
        gradient = tf.gradients(error, [self.w])
        updated_w = tf.assign(self.w, self.w -learningrate*gradient[0])

        return updated_w

In [214]:
y = dftrain["SalePrice"]
x = dftrain["OverallQual"]
x = np.array([x, np.ones_like(x)], dtype = "float64")

modelo = LinearModel()

learning_rate = 0.01
epochs = 10000

tensor_x = tf.placeholder(tf.float32, [2,len(dftrain["OverallQual"])], "tensor_x")
tensor_y = tf.placeholder(tf.float32, [len(dftrain["SalePrice"])], "tensor_y")

prediction = modelo(tensor_x)
update_parameters = modelo.update(tensor_x, tensor_y, learning_rate)

with tf.train.MonitoredSession() as session:
    feed_dict = {tensor_x:x, tensor_y:y}
    
    for i in range(epochs+1):

        session.run(update_parameters, feed_dict = feed_dict)
        
        if (i)%1000 == 0:
            predictions = session.run(prediction, feed_dict = feed_dict)
            weights = session.run(modelo.w, feed_dict = feed_dict)
            cost = 1/2* len(dftrain) * np.mean((y - predictions.reshape(1168,))**2)
            print("Epoch: ", i, "Weights: ", weights, "Cost:", cost)
            print("-------------------------------------------------------------------------")

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Epoch:  0 Weights:  [[11880.685   1808.3193]] Cost: 9366731519285.186
-------------------------------------------------------------------------
Epoch:  1000 Weights:  [[ 35932.03  -34659.438]] Cost: 1530064957931.5703
-------------------------------------------------------------------------
Epoch:  2000 Weights:  [[ 39723.402 -58889.016]] Cost: 1458473372127.066
-------------------------------------------------------------------------
Epoch:  3000 Weights:  [[ 42062.7  -73838.77]] Cost: 1431218934494.2253
-------------------------------------------------------------------------
Epoch:  4000 Weights:  [[ 43506.062 -83062.914]] Cost: 1420843262236.494
-------------------------------------------------------------------------
Epoch:  5000 Weights:  [[ 44396.613 -88754.164]] Cost: 1416893332818.2869
-------------------------------------------------------------------------
